In [1]:
from utils.mapping import production_mapping
from utils.calculation import get_initial_data, get_data 
import pandas as pd
import numpy as np
import plotly.graph_objects as go

import dash
from dash import dcc, html, Input, Output

In [2]:
# def extend_df(df):
#     df = df.copy()
#     extended_dates = pd.date_range(start=df['period'].iloc[-1] + pd.Timedelta(days=7), end='2024-12-27', freq='7D')
#     extended_df = pd.DataFrame({'period': extended_dates})
#     df = pd.concat([df, extended_df], ignore_index=True)
#     return df

def add_weekOfYear_Year(df):
    df = df.copy()
    df.insert(0,'week_of_year',df['period'].dt.isocalendar().week)
    df.insert(1,'year',df['period'].dt.year)
    df = df[df['week_of_year'] <= 52]
    return df

def select_seasonality_range(df, range_selector):
    df = df.copy()
    
    if range_selector == '1519':
        years = [2015, 2016, 2017, 2018, 2019]
    elif range_selector == '1823':
        years = [2018, 2019, 2021, 2022, 2023]
    
    def get_min_max(df, min_or_max, years):
        df = df.copy()
        df = df[df['year'].isin(years)]
        df = df.drop(columns=['period', 'year'])
        if min_or_max == 'min':
            df = df.groupby(['week_of_year']).min().reset_index()
        elif min_or_max == 'max':
            df = df.groupby(['week_of_year']).max().reset_index()
        return df

    def get_average(df, years):
        df = df[df['year'].isin(years)]
        df = df.drop(columns=['period', 'year'])
        df = df.groupby(['week_of_year']).mean().reset_index()
        df.insert(0, 'type', f'average_{range_selector}')    
        return df

    df_min = get_min_max(df, 'min', years)
    df_min.insert(0, 'type', f'min_{range_selector}')
    
    df_max = get_min_max(df, 'max', years)
    df_max.insert(0, 'type', f'max_{range_selector}')
        
    df_average = get_average(df, years)
    
    dff = pd.concat([df_min, df_max,df_average])
    
    return dff


def get_seasonality_data(df):
    df = df.copy()
    # df = extend_df(df)
    df = add_weekOfYear_Year(df)

    df_1519 = select_seasonality_range(df, '1519')
    df_1823 = select_seasonality_range(df, '1823')    

    df.insert(0, 'type', 'actual')
    df['type'] = df['type'] + '_' + df['year'].astype(str)
    df.drop(columns=['year'], inplace=True)

    df = pd.concat([df,df_1519, df_1823])    
    df['week_of_year'] = df['week_of_year'].astype(int)
    
    df['period'] = pd.to_datetime(df['period'])
    
    
    def format_date(date):
        return date.strftime('%b %d') if pd.notnull(date) else 'No Date'    
    df['period'] = df['period'].apply(format_date)
    
    def format_value_case_insensitive(val, column_name):
        if "stocks" in column_name.lower():
            val = val/1000
            return val

    df = df.set_index(['type','week_of_year','period'])
    cols = df.columns
    cols = [production_mapping[col] for col in cols]
    df.columns = cols

    for column in df.columns:
        df[column] = df[column].apply(lambda x: format_value_case_insensitive(x, column))

    reversed_mapping = {v: k for k, v in production_mapping.items()}
    df.columns = [reversed_mapping[col] for col in df.columns]
    
    df = df.reset_index()
    

    return df



In [3]:
df = get_seasonality_data(get_initial_data())

In [4]:
df['type'].unique() 

array(['actual_2015', 'actual_2016', 'actual_2017', 'actual_2018',
       'actual_2019', 'actual_2020', 'actual_2021', 'actual_2022',
       'actual_2023', 'actual_2024', 'min_1519', 'max_1519',
       'average_1519', 'min_1823', 'max_1823', 'average_1823'],
      dtype=object)

In [5]:
def pivot_individual_data(df,id):
    
    def get_individual_data(df, id):
        cols = ['type','week_of_year','period', id] 
        df = df[cols]
        df = df.rename(columns={id: 'value'})
        type_to_remove = ['actual_2015', 'actual_2016', 'actual_2017', 'actual_2018', 'actual_2019','actual_2020','actual_2021']
        df = df[~df['type'].isin(type_to_remove)]        
        return df    
    
    df = df.copy()
    df = get_individual_data(df, id)

    pivot_values = df.pivot_table(index=['week_of_year'], columns='type', values='value', aggfunc='first')
    pivot_dates = df[df['type'].str.contains('actual')].pivot_table(index=['week_of_year'], columns='type', values='period', aggfunc='first')
    pivot_dates.columns = [f'dates_{col}' for col in pivot_dates.columns]
    df = pd.concat([pivot_dates, pivot_values], axis=1)
    return df



ids = list(production_mapping.keys())
ids = ids[:50]

df = get_seasonality_data(get_initial_data())
dfs = {}
for id in ids:
    # df = pivot_individual_data(raw, id)
    dfs = {**dfs, **{id: pivot_individual_data(df, id)}}

In [6]:
tickvals = [0, 5, 9, 14, 18, 22, 27, 31, 36, 40, 45, 49]
ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

def chart_seasonality(df, id, toggle_seag_range, toggle_2022,toggle_2023,toggle_2024):

    def format_value(val, column_name):
        if "stocks" in column_name.lower():
            return f"{val:,.1f}" if val >= 0 else f"({abs(val):,.1f})"
        else:
            return f"{val:,.0f}" if val >= 0 else f"({abs(val):,.0f})"


    mapping_name = production_mapping[id]
    if 'stocks' in mapping_name.lower():
        mapping_name = mapping_name.replace('(kb)', '(mb)')

    df = df[id]

    if not toggle_seag_range:
        min_rng = df['min_1519']
        max_rng = df['max_1519']
        avg_rng = df['average_1519']
    else:
        min_rng = df['min_1823']
        max_rng = df['max_1823']
        avg_rng = df['average_1823']    


    traces = []

    traces.append(go.Scatter(
        x=df.index,
        y=min_rng,
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ))

    traces.append(go.Scatter(
        x=df.index,
        y=max_rng,
        mode='lines',
        fill='tonexty',
        fillcolor='#f4f5f9',
        line_color='#f4f5f9', 
        showlegend=False,
        hoverinfo='skip'
    ))

    traces.append(go.Scatter(
        x=df.index,
        y=avg_rng,
        mode='lines',
        line=dict(color='black', dash='dot', width=1),
        showlegend=False,
        hoverinfo='skip'
    ))

    if not toggle_2024:
        traces.append(go.Scatter(
            x=df.index,
            y=df['actual_2024'],
            mode='lines',
            name='2024',
            hoverinfo='text',
            text=[f"{date}: {format_value(value, mapping_name)}" for date, value in zip(df['dates_actual_2024'], df['actual_2024'])],
            line=dict(color='#c00000', dash='solid', width=2),
        ))

    if not toggle_2023:
        traces.append(go.Scatter(
            x=df.index,
            y=df['actual_2023'],
            mode='lines',
            name='2023',
            hoverinfo='text',
            line=dict(color='#e97132', dash='solid', width=2),
            text=[f"{date}: {format_value(value, mapping_name)}" for date, value in zip(df['dates_actual_2023'], df['actual_2023'])],
        ))

    if not toggle_2022:
        traces.append(go.Scatter(
            x=df.index,
            y=df['actual_2022'],
            mode='lines',
            name='2022',
            hoverinfo='text',
            line=dict(color='#bfbec4', dash='solid', width=2),
            text=[f"{date}: {format_value(value, mapping_name)}" for date, value in zip(df['dates_actual_2022'], df['actual_2022'])],
        ))


    layout = go.Layout(

        title=dict(
            text=mapping_name,
            y=0.97,
            x=0.03,
            xanchor='left',
            yanchor='top',
            font=dict(
                color='black'
            )
        ),   

        xaxis=dict(
            range=[0, len(df.index) - 1],
            tickmode='array',
            tickvals=tickvals,
            ticktext=ticktext,
            type='category',
            showspikes=True,
            spikedash='solid',
            spikecolor='grey',
            spikethickness=0.5,
            spikemode='across',
            showline=True,
            showgrid=False,
            linecolor='black',
            linewidth=0.5,
            zeroline=False
        ),
        yaxis=dict(
            tickformat=',.0fK',
            showgrid=False,
            showline=True,
            linecolor='black',
            linewidth=0.5,
            zeroline=False
        ),
        hovermode='x unified',
        template='plotly_white',
        legend=dict(
            x=0.85,
            y=1.0,
            xanchor='left',
            yanchor='top',
            orientation='v'
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Open Sans",
            bordercolor="#ccc"
        ),
        margin=dict(l=40, r=40, t=40, b=25),        
        showlegend=True
    )


    # Return the figure as a dictionary
    return {'data': traces, 'layout': layout}

In [8]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='seasonality-chart'),
    dcc.Checklist(
        id='toggle-seag-range',
        options=[{'label': 'Toggle Range', 'value': 'toggle'}],
        value=[]
    ),
    dcc.Checklist(
        id='toggle-2022',
        options=[{'label': 'Show 2022', 'value': 'toggle'}],
        value=['toggle']
    ),
    dcc.Checklist(
        id='toggle-2023',
        options=[{'label': 'Show 2023', 'value': 'toggle'}],
        value=['toggle']
    ),
    dcc.Checklist(
        id='toggle-2024',
        options=[{'label': 'Show 2024', 'value': 'toggle'}],
        value=['toggle']
    )
])


@app.callback(
    Output('seasonality-chart', 'figure'),
    Input('toggle-seag-range', 'value'),
    Input('toggle-2022', 'value'),
    Input('toggle-2023', 'value'),
    Input('toggle-2024', 'value')
)
def update_chart(toggle_seag_range, toggle_2022, toggle_2023, toggle_2024):
    toggle_seag_range = bool(toggle_seag_range)
    toggle_2022 = bool(toggle_2022)
    toggle_2023 = bool(toggle_2023)
    toggle_2024 = bool(toggle_2024)
    return chart_seasonality(dfs, 'WCESTUS1', toggle_seag_range, toggle_2022, toggle_2023, toggle_2024)

if __name__ == '__main__':
    app.run_server(debug=True,port=8060)